In [1]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import cProfile
import wget
import gzip
import urllib
import zipfile
from io import StringIO
import smart_open

In [2]:
host_path = "https://dumps.wikimedia.org/other/pagecounts-raw/"

#list of years to loop over: 2008-2014
years = range(2008,2015)
#list of months to loop over: 01 - 12
months = ["%.2d" % i for i in range(1, 13)]
# month[-1]

In [3]:
# example of forming url:
def form_url(year, month,file_name=""):
    """ Returns url for specified year and month
    """
    return host_path+str(year)+"/"+str(year)+"-"+str(month)+"/"+file_name
form_url(2008,'01')

'https://dumps.wikimedia.org/other/pagecounts-raw/2008/2008-01/'

In [4]:
# url = host_path+str(year[0])+"/"+str(year[0])+"-"+str(month[0])

def get_files(url):
    """ Returns all files from given url as a list
    """
    resp = requests.get(url)
    soup = BeautifulSoup(resp.content,"html.parser")
    list_elements = soup.find_all("li")
    size = [li.text.split(" ")[-1].replace("M","") for li in list_elements]
#     print(size)
    list_elements = [li.a.get("href") for li in list_elements]
    return list_elements, size

# get_files(url)

# Save file names for every year

In [5]:
final_size = 0
for year in years:
    year_lst = []
    memory_lst = []
    for month in months:
        url = form_url(year, month)
        files, sizes = get_files(url)
#         print(len(files))
#         print(len(sizes))
        year_lst.append(files)
        memory_lst.append(sizes)
    year_lst = sum(year_lst, [])
    memory_lst = sum(memory_lst, [])
#     print(year_lst)
#     year_lst = [file for file in year_lst if file.startswith("pagecounts")] #!!!!
    
    df = pd.DataFrame({
        "file": year_lst,
        "size": memory_lst
    })
    df["filter"] = df.file.apply(lambda x: "yes" if x.startswith("pagecounts") else "no")
    df = df[df["filter"] == "yes"]
    df = df.drop("filter", axis=1)
    df["size"] = pd.to_numeric(df["size"], errors='coerce')
#     print(df)
    path = str(year)+".csv"
    df.to_csv(path,index=False)
    print("File saved:"+path)
    total = df["size"].sum()/1024
    print("Cumulative file size: {} in GB".format(total))
    print(df.shape)
    final_size = final_size + total
print("Total size to be downloaded {} in GB".format(final_size))

File saved:2008.csv
Cumulative file size: 353.807421875 in GB
(8762, 2)
File saved:2009.csv
Cumulative file size: 472.96064453125 in GB
(8510, 2)
File saved:2010.csv
Cumulative file size: 520.15537109375 in GB
(8616, 2)
File saved:2011.csv
Cumulative file size: 625.60751953125 in GB
(8760, 2)
File saved:2012.csv
Cumulative file size: 729.875 in GB
(8784, 2)
File saved:2013.csv
Cumulative file size: 791.52548828125 in GB
(8766, 2)
File saved:2014.csv
Cumulative file size: 830.06640625 in GB
(8756, 2)
Total size to be downloaded 4323.997851562501 in GB


In [6]:
pd.read_csv("2014.csv").head()

,file,size
0,pagecounts-20140101-000000.gz,82.0
1,pagecounts-20140101-010000.gz,83.0
2,pagecounts-20140101-020000.gz,79.0
3,pagecounts-20140101-030001.gz,79.0
4,pagecounts-20140101-040008.gz,79.0


In [7]:
# url = form_url(2008,'01',"pagecounts-20080101-010000.gz")

In [8]:
# import gzip
# def download_file(url):
#     local_filename = url.split('/')[-1]
#     # NOTE the stream=True parameter
#     r = requests.get(url, stream=True)
#     with gzip.open(r, 'rb') as f:
#         for chunk in r.iter_content(chunk_size=1024): 
#             print(chunk)
#             if chunk: # filter out keep-alive new chunks
#                 f.write(chunk)
#                 #f.flush() commented by recommendation from J.F.Sebastian
#     return local_filename

In [9]:
# download_file(url)

In [10]:
# import zlib

# def stream_gzip_decompress(url):
#     stream = requests.get(url, stream=True)
#     dec = zlib.decompressobj(32 + zlib.MAX_WBITS)  # offset 32 to skip the header
#     for chunk in stream:
#         rv = dec.decompress(chunk)
#         print(rv)
#         print("-------------")
#         if rv:
#             print(rv)
# stream_gzip_decompress(url)

In [11]:
# import zlib
# CHUNKSIZE = 1024

# data2 = zlib.decompressobj(wbits = zlib.MAX_WBITS)  
# file = open('files/pagecounts-20140701-040000.gz', 'rb')  
# buf = file.read(CHUNKSIZE)
# # print(buf)
# # Decompress stream chunks
# while buf:  
#     repr(buf[:30])
#     decompressed_data = data2.decompress(buf)
    
#     print(decompressed_data)
#     buf = file.read(CHUNKSIZE)
#     print(buf)

# decompressed_data += data2.flush()

In [12]:
# f_path = 'files/pagecounts-20140701-040000.gz'

In [13]:
# import gzip

# input_file = gzip.open(f_path, 'rb')
# try:
#     print(input_file.read())
# finally:
#     input_file.close()

In [14]:
# def download_file(file_name, save_path):
#     date = file_name.split("-")[1]
#     year = date[:4]
#     month = date[4:6]
#     url = form_url(year, month, file_name)
#     print(url)
#     response = requests.get(url)
#     print(response.content)

In [15]:
# download_file("pagecounts-20140701-040000.gz", "files/")

In [16]:
# import smart_open
# file_name = "pagecounts-20140701-040000.gz"
# date = f_path.split("-")[1]
# year = date[:4]
# month = date[4:6]
# url = form_url(year, month, file_name)
# i = 0
# for line in smart_open.smart_open(url):
# #     line = line.decode("utf-8")
# #     if  line[:2] == "en" and i<100:
#     i = i+1
#     if i > 400000:
#         print(line[:2])
#     if i > 400100:
#         break

In [17]:
# x = "Str ".encode('utf-8')
# y = "Str".encode('utf-8')
# x.split(" ".encode('utf-8'))

In [18]:
# project = "en"
# project = project.encode('utf-8')

In [19]:
# https://stackoverflow.com/questions/19602931/basic-http-file-downloading-and-saving-to-disk-in-python

In [20]:
# def download_wget(file_name, save_path):
#     """ Downloads .gz file using the wget lib, saves file at specified path
#     """
#     date = file_name.split("-")[1]
#     year = date[:4]
#     month = date[4:6]
#     url = form_url(year, month, file_name)
#     print(url)
#     save = save_path+"/"+file_name
#     wget.download(url, save)

# Downloading the .gz file

In [21]:
file_name = "pagecounts-20140701-040000.gz"
save_path = "files/"

In [22]:
def download(file_name, save_path, lib="wget"):
    """ Downloads .gz file using the specified library and saves it on the specified path
    """
    # extract date information from file name and construct a url
    date = file_name.split("-")[1]
    year = date[:4]
    month = date[4:6]
    url = form_url(year, month, file_name)
    print(url)
    # saving location
    save = save_path+"/"+file_name
    
    if lib == "wget":
            wget.download(url, save)
    if lib == "requests": 
        response = requests.get(url, stream=True)
        save = save_path+"/"+file_name
        handle = gzip.open(save, "wb")
        for chunk in response.iter_content(chunk_size=512):
#             if chunk:  # filter out keep-alive new chunks
                handle.write(chunk)
    if lib == "urllib":
        testfile = urllib.request.urlretrieve(url,save)
    if lib == "smart_open":
        handle = gzip.open(save, "wb")
        for line in smart_open.smart_open(url):
            handle.write(line)

## Requests

In [24]:
cProfile.run('download(file_name, save_path, "requests")')

https://dumps.wikimedia.org/other/pagecounts-raw/2014/2014-07/pagecounts-20140701-040000.gz
         5990632 function calls (5990631 primitive calls) in 225.822 seconds

   Ordered by: standard name

   ncalls  tottime  percall  cumtime  percall filename:lineno(function)
        1    0.000    0.000    0.000    0.000 <frozen importlib._bootstrap>:996(_handle_fromlist)
        1    0.384    0.384  225.822  225.822 <ipython-input-22-7824cb84d7de>:1(download)
        1    0.000    0.000    0.000    0.000 <ipython-input-3-a5885820ab7c>:2(form_url)
        1    0.000    0.000  225.822  225.822 <string>:1(<module>)
        9    0.000    0.000    0.000    0.000 <string>:12(__new__)
        2    0.000    0.000    0.000    0.000 __init__.py:1257(debug)
        2    0.000    0.000    0.000    0.000 __init__.py:1501(getEffectiveLevel)
        2    0.000    0.000    0.000    0.000 __init__.py:1515(isEnabledFor)
        1    0.000    0.000    0.000    0.000 _collections.py:135(__init__)
   163205   

# Wget

In [25]:
cProfile.run('download(file_name, save_path)')

https://dumps.wikimedia.org/other/pagecounts-raw/2014/2014-07/pagecounts-20140701-040000.gz
         348912 function calls (348882 primitive calls) in 54.525 seconds

   Ordered by: standard name

   ncalls  tottime  percall  cumtime  percall filename:lineno(function)
    30603    0.037    0.000    0.156    0.000 <frozen importlib._bootstrap>:996(_handle_fromlist)
        1    0.000    0.000   54.525   54.525 <ipython-input-22-7824cb84d7de>:1(download)
        1    0.000    0.000    0.000    0.000 <ipython-input-3-a5885820ab7c>:2(form_url)
        1    0.000    0.000   54.525   54.525 <string>:1(<module>)
        1    0.000    0.000    0.000    0.000 <string>:12(__new__)
        1    0.000    0.000    0.000    0.000 _collections_abc.py:599(__contains__)
        2    0.000    0.000    0.000    0.000 _collections_abc.py:611(items)
        2    0.000    0.000    0.000    0.000 _collections_abc.py:631(__init__)
       66    0.000    0.000    0.000    0.000 _collections_abc.py:675(__iter__)

## Urllib

In [26]:
cProfile.run('download(file_name, save_path, "urllib")')

https://dumps.wikimedia.org/other/pagecounts-raw/2014/2014-07/pagecounts-20140701-040000.gz
         185736 function calls in 41.123 seconds

   Ordered by: standard name

   ncalls  tottime  percall  cumtime  percall filename:lineno(function)
        1    0.000    0.000   41.123   41.123 <ipython-input-22-7824cb84d7de>:1(download)
        1    0.000    0.000    0.000    0.000 <ipython-input-3-a5885820ab7c>:2(form_url)
        1    0.000    0.000   41.123   41.123 <string>:1(<module>)
        1    0.000    0.000    0.000    0.000 <string>:12(__new__)
        1    0.000    0.000    0.000    0.000 _collections_abc.py:599(__contains__)
        2    0.000    0.000    0.000    0.000 _collections_abc.py:611(items)
        2    0.000    0.000    0.000    0.000 _collections_abc.py:631(__init__)
       66    0.000    0.000    0.000    0.000 _collections_abc.py:675(__iter__)
        8    0.000    0.000    0.000    0.000 _policybase.py:277(_sanitize_header)
        9    0.000    0.000    0.000   

# Smart Open

In [27]:
cProfile.run('download(file_name, save_path, "smart_open")')

https://dumps.wikimedia.org/other/pagecounts-raw/2014/2014-07/pagecounts-20140701-040000.gz
         78595961 function calls (78595960 primitive calls) in 146.475 seconds

   Ordered by: standard name

   ncalls  tottime  percall  cumtime  percall filename:lineno(function)
        1    0.000    0.000    0.000    0.000 <frozen importlib._bootstrap>:996(_handle_fromlist)
        1    5.950    5.950  146.473  146.473 <ipython-input-22-7824cb84d7de>:1(download)
        1    0.000    0.000    0.000    0.000 <ipython-input-3-a5885820ab7c>:2(form_url)
        1    0.000    0.000  146.475  146.475 <string>:1(<module>)
        9    0.000    0.000    0.000    0.000 <string>:12(__new__)
        3    0.000    0.000    0.000    0.000 __init__.py:1257(debug)
        3    0.000    0.000    0.000    0.000 __init__.py:1501(getEffectiveLevel)
        3    0.000    0.000    0.000    0.000 __init__.py:1515(isEnabledFor)
        1    0.000    0.000    0.000    0.000 _collections.py:135(__init__)
     8172 

# Total number of files to download

In [28]:
num_files = 0
for year in years:
    file_name = str(year)+".csv"
    df = pd.read_csv(file_name)
    num_files = num_files + len(df)
print("Number of files to be downloaded {}".format(num_files))

Number of files to be downloaded 60954
